# `20180403b` - server side  - version p

In [58]:
#!/usr/bin/python
import spidev
import RPi.GPIO as GPIO
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('Agg')

def CreateDACCurve(Deb,Fin,CurveType):
    n = 200/5
    DACValues = []
    for k in range(n+1):
        if CurveType:
            val = int(Deb+1.0*k*(Fin-Deb)/n)
        else:
            val = int((Fin-Deb)*k**3/n**3+Deb)
        DACValues.append(val) 
    DACValues[-1] = 0
    return DACValues,len(DACValues)
    
    
def SetDACCurve(spi,DACValues):
    if len(DACValues) < 42: # to correct
        for i in range(len(DACValues)):
            if (DACValues[i] >= 0) and (DACValues[i] < 1020):
                WriteFPGA(spi,16+i,DACValues[i]/4)
            else:
                 WriteFPGA(spi,16+i,0)
    return 0

def WriteFPGA(spi,adress,value):
    spi.xfer([0xAA] )
    spi.xfer([adress] )
    spi.xfer([value] )
    
def StartUp():
    GPIO.setmode(GPIO.BCM)
    PRESET = 25
    IO4 = 26
    GPIO.setup(PRESET,GPIO.OUT)
    GPIO.setup(IO4,GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
    print "Reset 25 - Low 1s"
    GPIO.output(PRESET,GPIO.LOW)
    time.sleep(3)
    print "Reset 25 - High 0.2s"
    GPIO.output(PRESET,GPIO.HIGH)
    time.sleep(0.2)

    spi = spidev.SpiDev()
    spi.open(0,1) # CS2 - FPGA, on CE1 = IO4
    spi.mode = 0b01
    print "spi.cshigh is " + str(spi.cshigh)
    print "spi mode is " + str(spi.mode)
    spi.max_speed_hz = 2000000
    print "spi maxspeed is "+str(spi.max_speed_hz)+"hz"
    return spi

def TestSPI(spi,ncycles):
    i = 0
    while i < ncycles:
        WriteFPGA(spi,0xEB,0x01) # 0: single mode 1 continious mode
        time.sleep(0.5)
        WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode
        time.sleep(0.5)  
        i = i+1

def LoopSPI(spi):
    while 1:
        WriteFPGA(spi,0xEB,0x01) # 0: single mode 1 continious mode
        WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode

def LoopAcq(spi):
    while 1:
        WriteFPGA(spi,0xEB,0x00) # Doing 1 shot 
        WriteFPGA(spi,0xEF,0x01) # Cleaning memory pointer
        WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software
        time.sleep(0.001) # sleep 1ms
    
def ClearMem(spi):
    WriteFPGA(spi,0xEF,0x01) # To access memory

def ConfigSPI(spi):
    # Setup FPGA values by default
    setPon(200,spi)              # Set PulseOn
    setPulsesDelay(100,spi)      # Set Lengh between Pon and Poff: 100ns
    setPoff(2000,spi)            # Setting Poff 2us
    #setDACConstant(20,spi)   # gain at 20mV (2%)
    WriteFPGA(spi,0xEC,0x33)
    setDeltaAcq(7000,spi)    # 7us
    #WriteFPGA(spi,0xEA,0x00) # Software Trig : As to be clear by software
    WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode
    WriteFPGA(spi,0xED,0x03) # Frequency of ADC acquisition / sEEADC_freq (3 = 16Msps, 1 = 32, 0 = 64, 2 = 21Msps)
    WriteFPGA(spi,0xEE,0xA0) # How many cycles in countinious mode
    print "Config FPGA done!"

def setDACConstant(mV,spi):
    if mV > 1000:
        mV = 1000
    elif mV < 0:
        mV = 0   
    hmV = mV/4
    print "Gain:", mV," mV -- ",hex(hmV)
    WriteFPGA(spi,0xEC,hmV) # Voltage gain control: 0V to 1V
    
def setPon(POn,spi):
    if POn > 2500:
        POn = 2500
    elif POn < 0:
        POn = 0
    HPon = POn* 128 / 1000
    print "Pulse width:", POn," ns -- ",hex(HPon)
    WriteFPGA(spi,0xE0,HPon) # set sEEPon
    
def setPulsesDelay(DeltaPP,spi):
# Set Lengh between Pon and Poff
    if DeltaPP > 2500:
        DeltaPP = 2500
    elif DeltaPP < 0:
        DeltaPP = 0
    HPP =DeltaPP * 128 / 1000
    #print  hex(HPP)
    print "Pulses delay:", DeltaPP," ns -- ",hex(HPP)
    WriteFPGA(spi,0xD0,HPP) # set sEEPon

def setPoff(sEEPoff,spi):
    # Sets the damping length.
    POff = sEEPoff * 128 / 1000
    #print sEEPoff,POff
    POffMSB, POffLSB = 0xFF00&POff/0xFF,0x00FF&POff 
    print "Poff:", sEEPoff," ns -- ",hex(POffMSB),hex(POffLSB)
    WriteFPGA(spi,0xE1,POffMSB) # set sEEPon MSB
    WriteFPGA(spi,0xE2,POffLSB) # set sEEPon LSB

    # Setting Poff to Acq delay sEEDelayACQ
def setDeltaAcq(DeltaAcq,spi):
    if DeltaAcq > 255*255:
        DeltaAcq = 254*254
    elif DeltaAcq < 0:
        DeltaAcq = 0
    hDA = DeltaAcq * 128 / 1000
    hDAMSB, hDALSB = hDA/255 , 0x00FF&hDA 
    print "Delay between:",DeltaAcq,"ns -- ", hex(hDAMSB),hex(hDALSB)
    WriteFPGA(spi,0xE3,hDAMSB) # set sEEPon MSB
    WriteFPGA(spi,0xE4,hDALSB) # set sEEPon LSB
    
def SetLengthAcq(LAcqI,spi):
    LAcq = LAcqI * 128 / 1000
    #print LAcq,hex(LAcq),hex(LAcqI)
    LAcqMSB, LAcqLSB = 0x00FF&LAcq/256 , 0x00FF&LAcq
    print "Acquisition length: ", LAcq, " ns -- ",hex(LAcqMSB),hex(LAcqLSB)
    WriteFPGA(spi,0xE5,LAcqMSB) # set sEEPon MSB
    WriteFPGA(spi,0xE6,LAcqLSB) # set sEEPon LSB

def setPeriodAcq(lEPeriod,spi):
    lEPNs = lEPeriod*128/1000 #ns
    EPNsMSB, EPNs, EPNsLSB = 0x00FF&lEPNs/(256*256),0x00FF&lEPNs/256,0x0000FF&lEPNs 
    print "Period between two acquisitions:", lEPeriod,"us --", hex(EPNsMSB),hex(EPNs),hex(EPNsLSB) 
    WriteFPGA(spi,0xE7,EPNsMSB) # Period of one cycle MSB
    WriteFPGA(spi,0xE8,EPNs) # Period of one cycle 15 to 8
    WriteFPGA(spi,0xE9,EPNsLSB) # Period of one cycle LSB

def setPulseTrain(Pon,Pdelay,Poff,DelayAcq,Acq,spi):
    setPon(Pon,spi)
    setPulsesDelay(Pdelay+Pon,spi)
    setPoff(Poff+Pdelay+Pon,spi)
    setDeltaAcq(DelayAcq+Poff+Pdelay+Pon,spi)
    SetLengthAcq(Acq+DelayAcq+Poff+Pdelay+Pon,spi)
    #SetLengthAcq(Acq,spi)

In [59]:
spi = StartUp()
print("------")
ConfigSPI(spi)
print("------")
setPulseTrain(200,95,2000,7000,130000,spi)
N = 0

Reset 25 - Low 1s
Reset 25 - High 0.2s
spi.cshigh is False
spi mode is 1
spi maxspeed is 2000000hz
------
Pulse width: 200  ns --  0x19
Pulses delay: 100  ns --  0xc
Poff: 2000  ns --  0x0 0x0
Delay between: 7000 ns --  0x3 0x80
Config FPGA done!
------
Pulse width: 200  ns --  0x19
Pulses delay: 295  ns --  0x25
Poff: 2295  ns --  0x0 0x25
Delay between: 9295 ns --  0x4 0xa5
Acquisition length:  17829  ns --  0x45 0xa5


In [60]:
Curve = CreateDACCurve(200,200,False)[0] # Beginning, Ending, Linear (if False, expo)
print Curve
setDACConstant(0x22,spi)
#SetDACCurve(spi,Curve)


[200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 0]
Gain: 34  mV --  0x8


In [61]:
#LoopSPI(spi)
#LoopAcq(spi)

In [62]:
TestSPI(spi,3) # LED2 clignote 3x

In [63]:
N=0

In [67]:
f = 0x00
VGA = 0x22
#WriteFPGA(spi,0xEC,VGA)
LAcq = 130 #ns
WriteFPGA(spi,0xED,f) # Frequency of ADC acquisition / sEEADC_freq (3 = 16Msps, 1 = 32, 0 = 64, 2 = 21Msps)
WriteFPGA(spi,0xEB,0x01) # Doing one lines
WriteFPGA(spi,0xEF,0x01) # Cleaning memory pointer
WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software

time.sleep(0.2) # sleep 10ms

Fech = int(64/((1+f)))
Nacq = LAcq * Fech
print Fech, "-> "+str(Nacq) + ' samples' 

A = []
for i in range(2*Nacq):
    A.append ( spi.xfer([0x00] )[0] )  
a = np.asarray(A).astype(int)

np.savetxt( "p_A-"+str(LAcq)+"us-"+str(N)+"-VGA@"+str(hex(VGA))+"-spimode"+str(spi.mode)+"-"+str(Fech)+"msps.csv", a, delimiter=";")
N = N+1
print N

64 -> 8320 samples
2
